In [16]:
import gpc # library for work with GPS words over {0,1}
import math
import itertools
import re

## Work with $E_i$-palindromes and closures

In [2]:
def Ei(i):
    i = int(i)
    ei = [0,0,0]
    ei[i] = str(i)
    ei[(i+1)%3] = str((2+i)%3)
    ei[(i+2)%3] = str((1+i)%3)
    return ei

In [17]:
# Example:
print(Ei(1))

['2', '1', '0']


In [3]:
def isEipal(seq, i):
    """Checks if a string seq is an E_i palindrome."""
    ei = Ei(i)
    l = len(seq)
    if l == 1:
        if seq == str(i):
            return True
        else:
            return False
    for x in range(0, math.ceil(l/2)):
        if seq[x] != ei[int(seq[l-1-x])]:
            return False
    return(True)

In [23]:
# Example:
print(isEipal("012", 1))
print(isEipal("002", 1))
print(isEipal("01201", 2))

True
False
True


In [4]:
def testPalindromicity(seq):
    """Checks if a seq is an palindrome or and E-palindrome and 
    returns its nature."""
    if isEipal(seq,0):
        return [True, "0"]
    elif isEipal(seq, 1):
        return [True, "1"]
    elif isEipal(seq, 2):
        return [True, "2"]
    elif gpc.isPal(seq):
        return [True, "R"]
    else:
        return [False]

In [29]:
# Example:
print(testPalindromicity("0210"))
print(testPalindromicity("00")) # We want 00 to be an E_0-palindrome
print(testPalindromicity("010"))
print(testPalindromicity("02110"))

[True, '0']
[True, '0']
[True, 'R']
[False]


In [31]:
def makeEipalClosure (seq, i):
    """Makes E_i-th palindromic closure of a string."""
    ei = Ei(i)
    if isEipal(seq, i) == True:
        return(seq)
    j = 1
    while isEipal(seq[j:], i) != True:
        j = j+1
    gpc.verboseprint(2, "    {0} longest palindromic \
                     suffix : {1}".format(seq,seq[j:]))
    closure = seq
    pref = seq[j-1::-1]
    for letter in pref:
        closure = closure + ei[int(letter)]
    return(closure)

In [38]:
# Example:
print(makeEipalClosure("01", 1))
print(makeEipalClosure("00", 2))
print(makeEipalClosure("00", 0))
print(makeEipalClosure("021210", 2))

012
0011
00
0212102021


In [32]:
def make012Word(delta, theta, steps, seed = ""):
    """Makes a GPS over {0,1,2} from sequences delta and theta."""
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        gpc.verboseprint(1, "w{0} = {1}".format(step+1,w))
    return(w)

In [42]:
# Example:
make012Word("0012", "0020", 4)

'00112200'

## Naive function for controlling normalization

In [34]:
def is012NormalizedNaive(delta, theta, steps):
    """Checks if delta and theta are normalized and if not, 
    returns the beginning of the normalized sequence."""
    w = ""
    l=1
    prefixes = []
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = gpc.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        prefixes.append(w)
    gpc.verboseprint(1, "Prefixes from (delta, theta): " + str(prefixes))
    gpc.verboseprint(1, "Obtained word: " + w)
    newdelta = delta[0]
    newtheta = ""
    while l <= len(w):
        prefix = w[:l]
        res = testPalindromicity(prefix)
        if res[0] == True:
            if l< len(w):
                newdelta = newdelta + w[l]
            newtheta = newtheta + res[1]           
        l=l+1
    if newdelta == delta[:steps] and newtheta == theta[:steps]:
        return [True]
    else:
        return [False, newdelta, newtheta]
    
    # The length of newdelta and newtheta are the same since the whole word 
    # is an palindrome because if was generated by the GPS construction
    # from delta and theta.

In [41]:
# Example: 
print(is012NormalizedNaive("0012", "0020", 4))
print(is012NormalizedNaive("00120", "00210", 4))

[False, '00120', '00210']
[True]


## Implementation of the normalization algorithm

### Preprocessing of the normalized bi-sequence for our implementation

In our implementation of the algorithm, we decided to work only with words
that has 0 as first letter, 1 at second and 2 at third in order to make the 
algorithm easier to read and write. Of course, we want it working for all
the bi-sequences, so we have to preprocess the bi-sequence so that the word
obtained has first 0, then 1 and then 2. After the result of the algorithm,
we will go back to the original letters order.

In [65]:
def substitute(dic, seq):
    newseq = ""
    for l in seq:
        if l in dic:
            newseq = newseq + dic[l]
        else:
            newseq = newseq + l
    return newseq

In [66]:
dicti = {"0": "1", "2": "1"}
"1" in dicti

False

In [67]:
def changeLettersOrder(delta, theta):
    
    if delta[0] != 0:
        subs1 = {delta[0]: "0", "0": delta[0]}
        delta = substitute(subs, delta)
        theta = substitute(subs, delta)
    i = 0
    while delta[i] == "0":
        if theta[i] == "2":
            #konec
        if theta[i] == "1":
            subs3 = {"2": "1", "1": "2"}
        #jinak musime dal
        i = i + 1
    return [delta, theta, finalsubstitution]

In [68]:
changeLettersOrder("111122220000", "11111")

{'1': '0', '0': '1'}


'000022221111'

### Preprocessing of the beginning of the bi-sequence

We want that every word beginning with $i^l$ (where $l$ is the largest
possible) has a bi-directive sequence $(\Delta, \Theta)$ where the prefix
of $\Theta$ of lenght $l$ is equal to $E^l_i$. (We only solve the cases
when there is an $R$ instead of $E_0$ (e.g. $(0000, RE_0RE_0) \to (0000, E_0E_0E_0E_0)$, since the other cases are then solved
within the normalization process.

In [8]:
def beginningNormalization(delta, theta):
    biseq = gpc.makeBiseq(delta, theta)
    m = re.match("(0(R|0))+", biseq)
    if m:
        biseq = "00"*int((m.end()-m.start())/2) + biseq[m.end():]
    return gpc.parseBiseq(biseq)

In [9]:
beginningNormalization("000011", "R0R021")

['000011', '000021']

### Auxiliary functions

<img src="prefixesrules.jpg",width=600>
    
    
    

In [10]:
# List: bad prefix regex --> the new symbols instead of the last one
bad_prefixes = [
    ["(00)*02", "0012"], # 1.
    ["0012(0R12)*0R10", "1220"], # 2.
    ["0012(0R12)+01", "0R21"],  # 3.
    ["00121121", "2001"], # 4.
    ["001210", "1120"], # 5.
    ["001212", "1R02"], # 6.
    ["0012211R(111R)*1112", "1R02"] # 7.
    ["0012211R(111R)+10", "1100"] # 8.
    ["011R", "00120R"] # 9. !! the last two letters are rewritten !!
    ["001222", "210012"] # 10.
    ["0011", "1221"] # 11.
    ["0012(0R12)*10", "1R20"] # 12.
    ["0012(0R12)*0R11", "1221"] # 13.
    ["(001221)+00122R", "211R"] # 14.
    # 13 more to write down !
]

### Normalization algorithm

In [ ]:
def normalize(delta, theta):
    """Returns the normalized directive bi-sequence giving the same GPS word
    as (delta, theta)"""
    # Normalization of the letters order
    
    
    # Normalization of the prefix
    [delta, theta] = beginningNormalization(delta, theta)
    
    biseq = makeBiseq(delta, theta)
    bad_matches = find_bad_factors()
    
    # The main algorithm